In [1]:
import zipfile
import itertools
import tempfile
import math


# !pip install pygraphviz

from functools import reduce
from pyvis import network as net

import pandas as pd
import numpy as np
import networkx as nx
import plotly.express as px
import plotly.graph_objects as go

from plotly.offline import iplot
from IPython.display import display, HTML


pd.options.plotting.backend = "plotly"

#for Jupiter notebooks
import plotly.io as pio #comment for Google collab
pio.renderers.default='notebook'#comment for Google collab
from plotly.offline import init_notebook_mode #for Google collab comment these two lines 
init_notebook_mode(connected=True) #for Google collab comment these two lines 

In [2]:
df = pd.read_csv('var_4_2.csv',encoding="latin-1", sep=';')
df.columns = [ 
    'User ID (sender)', 
    'User ID (receiver)',
    'User account ID (sender)',
    'User account ID (receiver)',
    'Amount of transaction',
    'Type of transaction',
    'State of operation',
    'Balance before (sender)',
    'Balance after (sender)',
    'Balance after (receiver)',
    'Balance before (receiver)',    
    'Not used',
    'Not used',
    'Not used',
    'Not used',
    'Transaction timestamp (sender)',
    'Transaction timestamp (receiver)',
    'Sender account ID',
    'Not used',
    'Not used',
    'Not used',
    'Transaction timestamp',
    'Sender type',
    'Receiver type'
]
df

,User ID (sender),User ID (receiver),User account ID (sender),User account ID (receiver),Amount of transaction,Type of transaction,State of operation,Balance before (sender),Balance after (sender),Balance after (receiver),...,Not used,Transaction timestamp (sender),Transaction timestamp (receiver),Sender account ID,Not used,Not used,Not used,Transaction timestamp,Sender type,Receiver type
0,PN_EU_1_502,operator,EUAcc1_502,A0,2054.46,ArRC,SU,1.000000e+08,9.999795e+07,9.918004e+07,...,NaN,01.06.2011 0:16,01.06.2011 0:16,EUAcc1_502,NaN,NaN,Rc201161.01616,01.06.2011 0:16,EU,operator
1,PN_Ret2,PN_EU_1_362,RAcc2,EUAcc1_362,182548.74,Dt,SU,9.998681e+08,9.996855e+08,1.001825e+08,...,NaN,01.06.2011 0:36,01.06.2011 0:36,RAcc2,NaN,NaN,Dp201161.03636,01.06.2011 0:36,RET,EU
2,PN_EU_1_26,operator,EUAcc1_26,A0,5507.28,ArRC,SU,1.000000e+08,9.999449e+07,9.918554e+07,...,NaN,01.06.2011 0:36,01.06.2011 0:36,EUAcc1_26,NaN,NaN,Rc201161.03636,01.06.2011 0:36,EU,operator
3,PN_EU_1_18,operator,EUAcc1_18,A0,2102.6,ArRC,SU,1.000000e+08,9.999790e+07,9.918765e+07,...,NaN,01.06.2011 1:01,01.06.2011 1:01,EUAcc1_18,NaN,NaN,Rc201161.111,01.06.2011 1:01,EU,operator
4,PN_EU_0_786,operator,EUAcc0_786,A0,1361.77,ArRC,SU,1.000000e+08,9.999864e+07,9.918901e+07,...,NaN,01.06.2011 1:05,01.06.2011 1:05,EUAcc0_786,NaN,NaN,Rc201161.155,01.06.2011 1:05,EU,operator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54016,PN_EU_0_1833,PN_Ret3,EUAcc0_833,RAcc3,10942.82,Wl,SU,9.997550e+07,9.996445e+07,8.615736e+08,...,NaN,30.09.2011 23:30,30.09.2011 23:30,EUAcc0_116,NaN,NaN,Wt2011930.65454,30.09.2011 6:54,EU,RET
54017,PN_EU_0_1833,PN_Ret4,EUAcc0_833,RAcc4,883.07,Wl,SU,9.996445e+07,9.996355e+07,8.478874e+08,...,NaN,30.09.2011 23:32,30.09.2011 23:32,EUAcc0_1256,NaN,NaN,Wt2011930.85252,30.09.2011 8:52,EU,RET
54018,PN_EU_0_727,PN_EU_1_1193,EUAcc0_727,EUAcc0_1183,13778.82,Ind,SU,9.971685e+07,9.970294e+07,9.999935e+07,...,NaN,30.09.2011 23:33,30.09.2011 23:33,RAcc6,NaN,NaN,C2C2011930.113232,30.09.2011 11:32,EU,EU
54019,PN_EU_0_986,PN_EU_2_784,EUAcc0_986,EUAcc0_748,27614.07,Ind,SU,9.981849e+07,9.979060e+07,9.989427e+07,...,NaN,30.09.2011 23:38,30.09.2011 23:38,EUAcc1_197,NaN,NaN,C2C2011930.124242,30.09.2011 12:42,EU,EU


In [3]:
df = df.loc[:, ~df.columns.str.contains('^Not used', case=False)].sort_values('Transaction timestamp') 
df = df.drop('State of operation', axis=1)
df = df.drop('Sender account ID', axis=1)
df = df.drop('Transaction timestamp (sender)', axis=1)
df = df.drop('Transaction timestamp (receiver)', axis=1)
df = df.drop('Balance before (sender)', axis=1)
df = df.drop('Balance after (sender)', axis=1)
df = df.drop('Balance before (receiver)', axis=1)
df = df.drop('Balance after (receiver)', axis=1)

df.describe(include='all', datetime_is_numeric=True).fillna('')

,User ID (sender),User ID (receiver),User account ID (sender),User account ID (receiver),Amount of transaction,Type of transaction,Transaction timestamp,Sender type,Receiver type
count,54021,54021,54021,54021,54021,54021,54021,54021,54021
unique,1864,1562,1862,1562,53284,5,46388,2,4
top,PN_Ret4,operator,RAcc4,A0,2770.29,ArRC,08.07.2011 15:16,EU,operator
freq,2256,27901,2256,27901,4,27901,5,41238,27901


In [4]:
df['Amount of transaction']=df['Amount of transaction'].astype(str)
df['Amount of transaction'] = df['Amount of transaction'].str.replace(',','.')
df['Amount of transaction']=df['Amount of transaction'].astype(float)
df.head()

,User ID (sender),User ID (receiver),User account ID (sender),User account ID (receiver),Amount of transaction,Type of transaction,Transaction timestamp,Sender type,Receiver type
0,PN_EU_1_502,operator,EUAcc1_502,A0,2054.46,ArRC,01.06.2011 0:16,EU,operator
1,PN_Ret2,PN_EU_1_362,RAcc2,EUAcc1_362,182548.74,Dt,01.06.2011 0:36,RET,EU
2,PN_EU_1_26,operator,EUAcc1_26,A0,5507.28,ArRC,01.06.2011 0:36,EU,operator
71,PN_Ret4,PN_EU_1_233,RAcc4,EUAcc1_233,202013.28,Dt,01.06.2011 10:20,RET,EU
70,PN_EU_2_12,PN_Ret5,EUAcc2_12,RAcc5,330382.84,Wl,01.06.2011 10:20,EU,RET


In [5]:
df.dtypes

User ID (sender)               object
User ID (receiver)             object
User account ID (sender)       object
User account ID (receiver)     object
Amount of transaction         float64
Type of transaction            object
Transaction timestamp          object
Sender type                    object
Receiver type                  object
dtype: object

In [6]:
def init_stat_dict():
    stat_dict = dict()
    transaction_types = {"Ind", "Wl", "Dt", "Merchant", "ArRC"} 
    for tran_type in transaction_types:
          amount_name = f"Sent_amount_{tran_type}"
          amount_median = f"Sent_amount_{tran_type}_median"
          amount_min = f"Sent_amount_{tran_type}_min"
          amount_max = f"Sent_amount_{tran_type}_max"
          tran_count = f"Sent_{tran_type}_count"
          rec_amount_name = f"Received_amount_{tran_type}"
          rec_amount_median = f"Received_amount_{tran_type}_median"
          rec_amount_min = f"Received_amount_{tran_type}_min"
          rec_amount_max = f"Received_amount_{tran_type}_max"
          rec_tran_count = f"Received_{tran_type}_count"
          
          stat_dict[amount_name] = 0
          stat_dict[amount_median] = 0
          stat_dict[amount_min] = 0
          stat_dict[amount_max] = 0
          stat_dict[tran_count] = 0
          stat_dict[rec_amount_name] = 0
          stat_dict[rec_amount_median] = 0
          stat_dict[rec_amount_min] = 0
          stat_dict[rec_amount_max] = 0
          stat_dict[rec_tran_count] = 0

    return stat_dict


def get_stat_df(df):
   sent_unique_users = df["User ID (sender)"].unique()
   received_unique_users = df["User ID (receiver)"].unique()
   unique_users = np.unique(np.concatenate((sent_unique_users,received_unique_users),0))
    #unique_users = pd.concat(sent_unique_users,received_unique_users).drop_duplicates().reset_index(drop=True)
   print(unique_users)
   stat_df = pd.DataFrame()
   stat_dict = init_stat_dict()
   transaction_types = {"Ind", "Wl", "Dt", "Merchant", "ArRC"}
   for user in unique_users:
       stat_dict = init_stat_dict() 
       stat_dict["User ID"] = user

       user_df = df.loc[(df["User ID (sender)"] == user)]
       
       if (not user_df.empty):
          #stat_dict["User ID"] = user
          
          stat_dict["Unique_receivers"] = len(user_df["User ID (receiver)"].unique())
          stat_dict["User type"] = user_df["Sender type"].unique()[0]

          for tran_type in transaction_types:
              amount_name = f"Sent_amount_{tran_type}"
              amount_median = f"Sent_amount_{tran_type}_median"
              amount_min = f"Sent_amount_{tran_type}_min"
              amount_max = f"Sent_amount_{tran_type}_max"
              tran_count = f"Sent_{tran_type}_count"
              stat_dict[amount_name] = (user_df.loc[user_df["Type of transaction"]==tran_type])["Amount of transaction"].sum()
              stat_dict[amount_median] = (user_df.loc[user_df["Type of transaction"]==tran_type])["Amount of transaction"].mean()
              stat_dict[amount_min] = (user_df.loc[user_df["Type of transaction"]==tran_type])["Amount of transaction"].min()
              stat_dict[amount_max] = (user_df.loc[user_df["Type of transaction"]==tran_type])["Amount of transaction"].max()
              stat_dict[tran_count] = (user_df.loc[user_df["Type of transaction"]==tran_type])["Amount of transaction"].count()
       else:
          stat_dict["User type"] = (df.loc[(df["User ID (receiver)"]==user)])["Receiver type"].unique()[0]

       user_df = df.loc[(df["User ID (receiver)"] == user)]
       if (not user_df.empty):
          stat_dict["Unique_senders"] = len(user_df["User ID (sender)"].unique())
          for tran_type in transaction_types:
              rec_amount_name = f"Received_amount_{tran_type}"
              rec_amount_median = f"Received_amount_{tran_type}_median"
              rec_amount_min = f"Received_amount_{tran_type}_min"
              rec_amount_max = f"Received_amount_{tran_type}_max"
              rec_tran_count = f"Received_{tran_type}_count"
              stat_dict[rec_amount_name] = (user_df.loc[user_df["Type of transaction"]==tran_type])["Amount of transaction"].sum()
              stat_dict[rec_amount_median] = (user_df.loc[user_df["Type of transaction"]==tran_type])["Amount of transaction"].median()
              stat_dict[rec_amount_min] = (user_df.loc[user_df["Type of transaction"]==tran_type])["Amount of transaction"].min()
              stat_dict[rec_amount_max] = (user_df.loc[user_df["Type of transaction"]==tran_type])["Amount of transaction"].max()
              stat_dict[rec_tran_count] = (user_df.loc[user_df["Type of transaction"]==tran_type])["Amount of transaction"].count()
       
       df_temp = pd.DataFrame([stat_dict])
       
       #df_temp.head()
       stat_df = pd.concat([stat_df, df_temp])
   stat_df = stat_df.fillna(0)
   return stat_df

In [7]:
stat_df = get_stat_df(df)
print(stat_df.shape)

['PN_EU_0_0' 'PN_EU_0_1' 'PN_EU_0_10' ... 'PN_Ret5' 'PN_Ret6' 'operator']
(2011, 54)


In [8]:
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from matplotlib.ticker import FormatStrFormatter
import plotly.express as px
import pygraphviz

In [11]:
#оставляем поля, связанные с переводами и снятиями и добавили число уникальных пользователей, это же бот сеть.

MobileBot_labels = ['Unique_receivers','Unique_receivers','Sent_Ind_count' ,'Sent_Wl_count', 'Received_Ind_count']

# а по этим полям будем пробовать найти пользователей с кражей телефона.
MobileTheft_labels = ['Sent_amount_Wl', 'Sent_amount_Wl_median', 'Sent_amount_Wl_min', 'Sent_amount_Wl_max', 'Sent_Wl_count']

x = stat_df[MobileBot_labels].values

# нормализуем значения
x = StandardScaler().fit_transform(x)

pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
print(f'Explained variance: {pca.explained_variance_ratio_}\tSum: {pca.explained_variance_ratio_.sum()}')


Explained variance: [0.40136991 0.32977881 0.19937569]	Sum: 0.930524408134177


In [10]:
principalDf = pd.DataFrame(data=principalComponents , columns=['PC1', 'PC2', 'PC3'])

fig = px.scatter(principalDf, x="PC1", y="PC2", color=stat_df['User type'],
                 size=stat_df['Unique_senders'].apply(lambda x: 1 if x == 0 else math.log(x,10) ), 
                 hover_name=stat_df['User ID'], 
                 
                 opacity = 0.5,
                 color_discrete_map={
                'EU': '#377eb8',
                'operator': '#e41a1c',
                'RET': '#4daf4a',
                'MER': '#984ea3',
                })

#for GOogle colab uncomment
pio.renderers.default = 'iframe' #comment this  lines for GOogle colab

fig.show()


In [12]:
fig = px.scatter(stat_df, x='Sent_amount_Wl_max',  y='Sent_amount_Wl_median', color=stat_df['User type'],
                 size=stat_df['Unique_senders'].apply(lambda x: 1 if x == 0 else math.log(x,10) ), 
                 hover_name=stat_df['User ID'], 
                 
                 opacity = 0.5,
                 color_discrete_map={
                'EU': '#377eb8',
                'operator': '#e41a1c',
                'RET': '#4daf4a',
                'MER': '#984ea3',
                })

fig.show()

In [13]:
def create_network_for_df(df):
    G = nx.MultiDiGraph()

    for _, row in df.iterrows():
        sender_node_id = row["User ID (sender)"]
        receiver_node_id = row["User ID (receiver)"]
        sent_tran_num = dict({'Ind': 0, 'Wl': 0, 'Dt':0, 'ArRC':0, 'Merchant':0})
        received_tran_num = dict({'Ind': 0, 'Wl': 0, 'Dt':0, 'ArRC':0, 'Merchant':0})
        sent_tran_amount = dict({'Ind': 0, 'Wl': 0, 'Dt':0, 'ArRC':0, 'Merchant':0})
        received_tran_amount = dict({'Ind': 0, 'Wl': 0, 'Dt':0, 'ArRC':0, 'Merchant':0})

        tran_type = row["Type of transaction"]
        if sender_node_id in G.nodes().keys():
            
            
            G.nodes[sender_node_id]['sent_transaction_num'][tran_type]+=1 
            G.nodes[sender_node_id]['sent_transaction_amount'][tran_type]+=row["Amount of transaction"] 
        else:
            sent_tran_num[tran_type]=1
            sent_tran_amount[tran_type] = row["Amount of transaction"]
            G.add_node(
              sender_node_id, 
              type=row["Sender type"], 
              account_id=row["User account ID (sender)"],
              sent_transaction_num = sent_tran_num,
              sent_transaction_amount = sent_tran_amount,
              received_transaction_num = dict({'Ind': 0, 'Wl': 0, 'Dt':0, 'ArRC':0, 'Merchant':0}),
              received_transaction_amount = dict({'Ind': 0, 'Wl': 0, 'Dt':0, 'ArRC':0, 'Merchant':0})
              )
        
        if receiver_node_id in G.nodes().keys():
            G.nodes[receiver_node_id]['received_transaction_num'][tran_type]+=1
            G.nodes[sender_node_id]['received_transaction_amount'][tran_type]+=row["Amount of transaction"]
        else:
            received_tran_num[tran_type]=1
            received_tran_amount[tran_type]=row["Amount of transaction"]
            G.add_node(
            receiver_node_id, 
            type=row["Receiver type"], 
            account_id=row["User account ID (receiver)"],
            transaction_num = 1,
            sent_transaction_num = dict({'Ind': 0, 'Wl': 0, 'Dt':0, 'ArRC':0, 'Merchant':0}),
            sent_transaction_amount = dict({'Ind': 0, 'Wl': 0, 'Dt':0, 'ArRC':0, 'Merchant':0}),
            received_transaction_num = received_tran_num,
            received_transaction_amount = received_tran_amount
        )

        G.add_edge(
            row["User ID (sender)"], 
            row["User ID (receiver)"], 
            amount=row["Amount of transaction"],
            timestamp=row["Transaction timestamp"],
            type=row["Type of transaction"],           
        )
    return G

In [14]:
def draw_palette(colors, shapes=None, title=""):

    if shapes:
        assert len(colors) == len(shapes)

    x = list(colors.keys())
    y = [1 for _ in x]
    color = list(colors.values())
    shape = None

    if shapes:
        shape = list(shapes.values())
    else:
        shape = ['circle' for _ in x]

    fig = go.Figure( 
        data=[
            go.Scatter(
                mode='markers',
                x=x, y=y, 
                marker=dict(size=40, 
                            color=color, 
                            symbol=shape,
                            line=dict(width=0, color='black')))
        ], 
        layout=dict(
            title_text=title, 
            yaxis=dict(visible=False, 
                       showticklabels=False)
        )
    )

    fig.show()

In [15]:
GROUNDTRUTH_TO_COLOR = {
    'N_RegC2C': '#666666', 
    'N_RegDep': '#666666', 
    'N_Reg_RC': '#666666', 
    'N_RegWith': '#666666',
    'N_Reg_Merch': '#666666',
    'F_Mule_With': '#990099',
    'F_bot': '#EECA3B',
    'F_SevWith': '#EF553B',
}
#палитра set 1 из ColorBrewer 
#e41a1c красный
#377eb8 синий
#4daf4a зеленый
#984ea3 фиолетовый
TYPE_TO_COLOR = {
    'EU': '#377eb8',
    'operator': '#e41a1c',
    'RET': '#4daf4a',
    'MER': '#984ea3',
}

TYPE_TO_SHAPE = {
    'EU': 'circle',
    'operator': 'square',
    'RET': 'diamond',
    'MER': 'star',
}

IS_CRIMINAL_TO_COLOR_SIZE_NODE = {
    True: ('#EF553B', 15, 3),
    False: ('gray', 15, 0)
}

IS_CRIMINAL_TO_COLOR_SIZE_EDGE = {
    True: ('#EF553B', 2),
    False: ('#666666', 1)
}

In [16]:
draw_palette(TYPE_TO_COLOR, TYPE_TO_SHAPE, title="User type palette")

In [17]:
def make_node(layout, node, data, is_criminal):
    x, y = layout[node]

    type_t = data['type']
    
    sent_tran_num = sum(data["sent_transaction_num"].values())
    received_tran_num = sum(data["received_transaction_num"].values())

    sent_tran_amount = sum(data["sent_transaction_amount"].values())
    received_tran_amount = sum(data["received_transaction_amount"].values())


    color, size, width = IS_CRIMINAL_TO_COLOR_SIZE_NODE[is_criminal]

    user = px.scatter(
        x=[x, None],
        y=[y, None],
        text = [f"{node}", None],
        hover_name=[f"{node}<br>"
              #f"<b>Criminal:</b> {is_criminal}<br>"
              f"Type: {type_t}<br>"
              f"Sent transactions (num)):{sent_tran_num}<br>"
              f"Received transactions (num):{received_tran_num}<br>"
              f"<b>Sent transactions (amount):</b>{sent_tran_amount}<br>"
              f"<b>Received transactions (amount):</b>{received_tran_amount}<br>"
              ,None]
              )
              
    
    user.update_traces(
        marker=dict(
            color=TYPE_TO_COLOR[type_t],
            size=size,  
            line=dict(
                width=width,
                color=color
            ),
            symbol=TYPE_TO_SHAPE[type_t]
        ),
        textposition='bottom center',
        textfont_size=8
    )

    return user
# 
def make_edge(layout, node_id1, node_id2, G, criminals):
    x0, y0 = layout[node_id1]
    x1, y1 = layout[node_id2]

    node1 = G.nodes[node_id1]  
    node2 = G.nodes[node_id2]

    is_sender_criminal = True if node_id1 in criminals else False
    is_receiver_criminal = True if node_id2 in criminals else False

    color, size = IS_CRIMINAL_TO_COLOR_SIZE_EDGE[is_sender_criminal]

    # Edge
    trace = px.line(
        x=[x0, x1, None], 
        y=[y0, y1, None],
        hover_data=None,
    )
    
    trace.update_traces(
        line_color=color,
        line_width=size,
        hovertemplate = None,
        hoverinfo = "skip",
    )

    node1_trace = make_node(layout, node_id1, node1, is_sender_criminal)
    node2_trace = make_node(layout, node_id2, node2, is_receiver_criminal)

    return trace.data, (node1_trace.data + node2_trace.data)


def draw_network(G, title, crime_type="", criminals=set()): 
    layout = nx.nx_agraph.graphviz_layout(G, prog = 'twopi')
   #полезными могут быть укладки dot и twopi (параметр prog)
   
   #Поэкспериментируйте с разными раскладками
   
    #layout = nx.spring_layout(G)
    #layout = nx.circular_layout(G)
    #layout = nx.kamada_kawai_layout(G, scale = 10 )
    

    edges_data = ()
    nodes_data = ()
    for node1, node2, _ in G.edges(data=True):
        edge, nodes = make_edge(layout, node1, node2, G, criminals)
        edges_data += edge
        nodes_data += nodes

    all_fig = go.Figure((*edges_data, *nodes_data))
    all_fig.update_layout(autosize=True, width=900, height=900)
    all_fig.write_html(f"{crime_type}_{title}_plotly.html")
    all_fig.show()



def draw_network_for_df(df, title, crime_type="", criminals=set()):
    G = create_network_for_df(df)
    draw_network(G, title, crime_type, criminals)

In [18]:
df = df.sort_values(by=['Transaction timestamp'], ascending=True)
suspected_users = ['PN_EU_0_1833', 'PN_EU_1_1193', 'PN_EU_2_784', 'PN_1_1855']
#suspected_users = ['PN_EU_2_154'] # постройте граф контактов для подозрительного пользователя
#suspected_users = ['PN_EU_0_955'] # постройте граф контактов для мула и сравните с подозрительным
df_selected = df.loc[((df["User ID (sender)"].isin(suspected_users)) | (df["User ID (receiver)"].isin(suspected_users)))]
print(df_selected.shape)

(1129, 9)


In [19]:
G = create_network_for_df(df_selected)

In [20]:
draw_network(G, "Структура мошенничества","Мобильный ботнет" )

In [54]:
# draw_network(G, "Структура мошенничества","Мобильный ботнетdot" )

In [61]:
# df = df.sort_values(by=['Transaction timestamp'], ascending=True)
# suspected_users = ['PN_EU_1_1855']
# df_selected = df.loc[((df["User ID (sender)"].isin(suspected_users)) | (df["User ID (receiver)"].isin(suspected_users)))]
# print(df_selected.shape)

(361, 9)


In [62]:
# G = create_network_for_df(df_selected)
# draw_network(G, "Структура мошенничества","Мобильный ботнет1855" )

In [21]:
suspected_users = ['PN_EU_0_1833', 'PN_EU_1_1193', 'PN_EU_2_784', 'PN_1_1855']
infected_users = (df.loc[((df["User ID (receiver)"].isin(suspected_users)) & (df['Type of transaction']=='Ind'))]["User ID (sender)"]).unique()
print(infected_users)

['PN_EU_0_507' 'PN_EU_0_323' 'PN_EU_0_1148' 'PN_EU_0_670' 'PN_EU_0_213'
 'PN_EU_0_722' 'PN_EU_0_1077' 'PN_EU_1_513' 'PN_EU_0_983' 'PN_EU_2_169'
 'PN_EU_1_4' 'PN_EU_0_233' 'PN_EU_3_7' 'PN_EU_0_812' 'PN_EU_0_487'
 'PN_EU_3_18' 'PN_EU_1_54' 'PN_EU_0_925' 'PN_EU_0_732' 'PN_EU_0_909'
 'PN_EU_0_986' 'PN_EU_0_534' 'PN_EU_1_516' 'PN_EU_0_464' 'PN_EU_0_99'
 'PN_EU_1_45' 'PN_EU_1_32' 'PN_EU_0_948' 'PN_EU_0_727' 'PN_EU_1_39'
 'PN_EU_0_1219' 'PN_EU_0_1092' 'PN_EU_1_276' 'PN_EU_1_517' 'PN_EU_1_97'
 'PN_EU_1_318' 'PN_EU_0_1158' 'PN_EU_1_262' 'PN_EU_0_588']


In [22]:
def draw_pyvis_network_for_df(df, title):
    nt = net.Network(
        height='900px', width='100%',
        heading=f"{title}",
        directed=True,
        notebook=True,

        cdn_resources='remote'
    )      

    for _, row in df.iterrows():
        
        
        nt.add_node(
            row["User ID (sender)"], 
            label=row["User ID (sender)"], 
            shape=TYPE_TO_SHAPE[row["Sender type"]], #имхо избыточно
            color=TYPE_TO_COLOR[row["Sender type"]],
            title=row["User ID (sender)"],
        )

        nt.add_node(
            row["User ID (receiver)"], 
            label=row["User ID (receiver)"], 
            shape=TYPE_TO_SHAPE[row["Receiver type"]],
            color= TYPE_TO_COLOR[row["Receiver type"]],
            #title=f"{'Criminal' if is_receiver_criminal else None}",
        )
# можно поэкспериментировать с цветом ребра в зависимости от размера перевода
        nt.add_edge(
            row["User ID (sender)"], 
            row["User ID (receiver)"],
            color='#666666',
            width=1,
            title=f'Amount: {row["Amount of transaction"]}',
            label=f"{row['Type of transaction']}: {row['Amount of transaction']}"
        )

    return nt

In [23]:
n = draw_pyvis_network_for_df(df_selected, "Mobile botnet infection")

In [24]:
n.toggle_physics(False)
#можно настраивать силы притяженяи и отталкивания
n.show_buttons(filter_=['physics'])
n.show("PN_EU_0_1833.html")

PN_EU_0_1833.html


In [71]:
n.show("PN_EU_1_1855.html")

PN_EU_1_1855.html


In [25]:
df['Transaction timestamp'] =pd.to_datetime(df['Transaction timestamp'])

In [26]:
df = df.sort_values(by=['Transaction timestamp'], ascending=True)
dfs_by_day = [g for _, g in df.groupby(df['Transaction timestamp'].dt.date)]
df_day = dfs_by_day[17] 

In [27]:
def get_theft_df(df):
    mean_amount = df["Amount of transaction"].mean()

    median_df = (
        df[['User ID (sender)', 'Type of transaction', 'Sender type']]
            .groupby(['User ID (sender)', 'Type of transaction', 'Sender type'])
            .size()
            .groupby(['Type of transaction', 'Sender type'])
            .median()
            .to_frame(name='Median number of transactions')
            .reset_index()
            .sort_values(by=['Sender type'], ascending=True))

    median_wl = median_df.loc[median_df['Type of transaction'] == 'Wl', 'Median number of transactions'].iloc[0]
    # Filter withdrawals with amount less than mean_amount
    theft_df = df[(df['Type of transaction'] == 'Wl') & (df["Amount of transaction"] < mean_amount)].copy()

    # Select users, who made more than median_number_of_transactions_by_type withdrawals
    suspected_users = theft_df.groupby(['User ID (sender)'])['User ID (sender)'].transform('count') > median_wl
    
    theft_df.insert(0, 'Suspected', False)
    theft_df.loc[:, 'Suspected'] = suspected_users

    suspected_senders_set = set(theft_df.loc[theft_df['Suspected'] == True, 'User ID (sender)'])
    suspected_receivers_set = set(theft_df.loc[theft_df['Suspected'] == True, 'User ID (receiver)'])

    criminals = (suspected_senders_set | suspected_receivers_set)

    return theft_df, criminals

In [28]:
theft_df, criminals = get_theft_df(df_day)
theft_df.size

60

In [29]:
G = create_network_for_df(theft_df)
draw_network(G, "Структура мошенничества","Кража мобильного телефона" )

In [30]:
df_suspected_thefts = df.loc[((df["Amount of transaction"] < 10000) & (df['Type of transaction']=='Wl'))]
df_suspected_thefts.head()

,User ID (sender),User ID (receiver),User account ID (sender),User account ID (receiver),Amount of transaction,Type of transaction,Transaction timestamp,Sender type,Receiver type
113,PN_EU_1_58,PN_Ret1,EUAcc1_58,RAcc1,5373.79,Wl,2011-01-06 14:28:00,EU,RET
52895,PN_EU_2_784,PN_Ret1,EUAcc0_748,RAcc1,9524.61,Wl,2011-01-07 00:49:00,EU,RET
11991,PN_EU_1_111,PN_Ret2,EUAcc1_111,RAcc2,7983.95,Wl,2011-01-07 01:18:00,EU,RET
12055,PN_EU_2_29,PN_Ret1,EUAcc2_29,RAcc1,586.11,Wl,2011-01-07 04:36:00,EU,RET
12260,PN_EU_2_148,PN_Ret2,EUAcc2_148,RAcc2,3877.60,Wl,2011-01-07 16:12:00,EU,RET


In [31]:
G = create_network_for_df(df_suspected_thefts)
draw_network(G, "Структура мошенничества","Кража мобильного телефона" )

In [37]:
suspected_theft = ['PN_EU_0_33']
df_selected = df.loc[((df["User ID (sender)"].isin(suspected_theft)) | (df["User ID (receiver)"].isin(suspected_theft)))]
df_selected.head()

,User ID (sender),User ID (receiver),User account ID (sender),User account ID (receiver),Amount of transaction,Type of transaction,Transaction timestamp,Sender type,Receiver type
1429,PN_EU_0_33,PN_EU_0_582,EUAcc0_33,EUAcc0_582,21113.62,Ind,2011-06-06 22:02:00,EU,EU
52557,PN_EU_0_33,PN_Ret1,EUAcc0_33,RAcc1,6039.31,Wl,2011-06-13 07:37:00,EU,RET
52558,PN_EU_0_33,PN_Ret3,EUAcc0_33,RAcc3,17646.20,Wl,2011-06-13 07:41:00,EU,RET
52559,PN_EU_0_33,PN_Ret2,EUAcc0_33,RAcc2,1310.67,Wl,2011-06-13 07:44:00,EU,RET
52560,PN_EU_0_33,PN_Ret4,EUAcc0_33,RAcc4,3744.70,Wl,2011-06-13 07:46:00,EU,RET


In [39]:
n = draw_pyvis_network_for_df(df_selected, "Mobile phone theft ")

In [41]:
n.toggle_physics(False)
#можно настраивать силы притяженяи и отталкивания
n.show_buttons(filter_=['physics'])
n.show("'PN_EU_0_33'.html")

'PN_EU_0_33'.html


In [60]:
suspected_theft = ['PN_EU_0_452']
df_selected = df.loc[((df["User ID (sender)"].isin(suspected_theft)) | (df["User ID (receiver)"].isin(suspected_theft)))]
df_selected.head()

,User ID (sender),User ID (receiver),User account ID (sender),User account ID (receiver),Amount of transaction,Type of transaction,Transaction timestamp,Sender type,Receiver type
52537,PN_EU_0_452,PN_Ret4,EUAcc0_452,RAcc4,16100.92,Wl,2011-03-06 05:13:00,EU,RET
52538,PN_EU_0_452,PN_Ret5,EUAcc0_452,RAcc5,16437.20,Wl,2011-03-06 05:16:00,EU,RET
52539,PN_EU_0_452,PN_Ret3,EUAcc0_452,RAcc3,15362.54,Wl,2011-03-06 05:19:00,EU,RET
52540,PN_EU_0_452,PN_Ret5,EUAcc0_452,RAcc5,2494.33,Wl,2011-03-06 05:24:00,EU,RET
4298,PN_EU_0_452,operator,EUAcc0_452,A0,804.27,ArRC,2011-06-13 18:15:00,EU,operator


In [59]:
df_suspected_transactions = df.loc[((df["User ID (sender)"] == "PN_EU_0_33") & df['User ID (receiver)'].isin(["PN_Ret1","PN_Ret2","PN_Ret3", "PN_Ret4"]))]
df_suspected_transactions

,User ID (sender),User ID (receiver),User account ID (sender),User account ID (receiver),Amount of transaction,Type of transaction,Transaction timestamp,Sender type,Receiver type
52557,PN_EU_0_33,PN_Ret1,EUAcc0_33,RAcc1,6039.31,Wl,2011-06-13 07:37:00,EU,RET
52558,PN_EU_0_33,PN_Ret3,EUAcc0_33,RAcc3,17646.20,Wl,2011-06-13 07:41:00,EU,RET
52559,PN_EU_0_33,PN_Ret2,EUAcc0_33,RAcc2,1310.67,Wl,2011-06-13 07:44:00,EU,RET
52560,PN_EU_0_33,PN_Ret4,EUAcc0_33,RAcc4,3744.70,Wl,2011-06-13 07:46:00,EU,RET


In [45]:
# Мне не настолько это интересно,чтобы повышать точность
df_suspected_thefts = df.loc[((df["Amount of transaction"] < 10000) & (df['Type of transaction']=='Wl'))]
df_suspected_thefts.head()

,User ID (sender),User ID (receiver),User account ID (sender),User account ID (receiver),Amount of transaction,Type of transaction,Transaction timestamp,Sender type,Receiver type
113,PN_EU_1_58,PN_Ret1,EUAcc1_58,RAcc1,5373.79,Wl,2011-01-06 14:28:00,EU,RET
52895,PN_EU_2_784,PN_Ret1,EUAcc0_748,RAcc1,9524.61,Wl,2011-01-07 00:49:00,EU,RET
11991,PN_EU_1_111,PN_Ret2,EUAcc1_111,RAcc2,7983.95,Wl,2011-01-07 01:18:00,EU,RET
12055,PN_EU_2_29,PN_Ret1,EUAcc2_29,RAcc1,586.11,Wl,2011-01-07 04:36:00,EU,RET
12260,PN_EU_2_148,PN_Ret2,EUAcc2_148,RAcc2,3877.60,Wl,2011-01-07 16:12:00,EU,RET


In [63]:
G = create_network_for_df(df_suspected_thefts)
draw_network(G, "Структура мошенничества","Кража мобильного телефона" )

In [64]:
n = draw_pyvis_network_for_df(df_suspected_thefts, "Mobile phone theft ")

In [65]:
n.toggle_physics(False)
#можно настраивать силы притяженяи и отталкивания
n.show_buttons(filter_=['physics'])
n.show("Mobile phone theft.html")

Mobile phone theft.html


In [66]:
victims =  df_suspected_thefts["User ID (sender)"].unique()
print(f'Число пользователей, у которых были украдены телефоны: {len(victims)}. В их число входят: {victims}')

Число пользователей, у которых были украдены телефоны: 88. В их число входят: ['PN_EU_1_58' 'PN_EU_2_784' 'PN_EU_1_111' 'PN_EU_2_29' 'PN_EU_2_148'
 'PN_EU_1_1193' 'PN_EU_1_97' 'PN_EU_0_135' 'PN_EU_2_47' 'PN_EU_0_1833'
 'PN_EU_2_19' 'PN_EU_2_110' 'PN_EU_1_1855' 'PN_EU_0_91' 'PN_EU_3_19'
 'PN_EU_0_336' 'PN_EU_0_323' 'PN_EU_0_452' 'PN_EU_0_364' 'PN_EU_0_22'
 'PN_EU_0_42' 'PN_EU_0_98' 'PN_EU_2_130' 'PN_EU_1_9' 'PN_EU_1_71'
 'PN_EU_2_156' 'PN_EU_1_145' 'PN_EU_0_110' 'PN_EU_0_53' 'PN_EU_1_235'
 'PN_EU_3_2' 'PN_EU_0_295' 'PN_EU_0_525' 'PN_EU_0_462' 'PN_EU_0_33'
 'PN_EU_2_69' 'PN_EU_3_36' 'PN_EU_3_1' 'PN_EU_2_72' 'PN_EU_1_133'
 'PN_EU_2_64' 'PN_EU_1_20' 'PN_EU_0_160' 'PN_EU_2_40' 'PN_EU_2_155'
 'PN_EU_3_28' 'PN_EU_2_75' 'PN_EU_1_86' 'PN_EU_3_5' 'PN_EU_1_98'
 'PN_EU_1_95' 'PN_EU_2_58' 'PN_EU_1_109' 'PN_EU_1_38' 'PN_EU_1_212'
 'PN_EU_2_60' 'PN_EU_1_16' 'PN_EU_2_14' 'PN_EU_1_76' 'PN_EU_2_157'
 'PN_EU_3_26' 'PN_EU_0_70' 'PN_EU_1_142' 'PN_EU_3_27' 'PN_EU_3_22'
 'PN_EU_3_32' 'PN_EU_3_24' 'PN_EU_1_16

In [94]:
df_selected = df.loc[((df["User ID (sender)"].isin(['PN_EU_1_58'])) | (df["User ID (receiver)"].isin(['PN_EU_1_58'])))]
df_selected=df_selected.loc[(df_selected['Type of transaction']=='Wl')]
(df_selected['Transaction timestamp'].iloc[-1] - df_selected['Transaction timestamp'].iloc[0])/df_selected.shape[0]

Timedelta('16 days 06:40:33.750000')

In [120]:
import datetime
df_selected = df.loc[((df["User ID (sender)"].isin(['PN_EU_0_33'])) | (df["User ID (receiver)"].isin(['PN_EU_0_33'])))]
df_selected= df_selected.loc[(df_selected['Type of transaction']=='Wl')]
(df_selected['Transaction timestamp'].iloc[-1] - df_selected['Transaction timestamp'].iloc[0])/df_selected.shape[0]

Timedelta('0 days 00:02:15')

In [125]:
# Ну эти точно воры
vic =[]
for v in victims:
    df_selected = df.loc[((df["User ID (sender)"].isin([v])) | (df["User ID (receiver)"].isin([v])))]
    df_selected= df_selected.loc[(df_selected['Type of transaction']=='Wl')]
    if (df_selected['Transaction timestamp'].iloc[-1] - df_selected['Transaction timestamp'].iloc[0])/df_selected.shape[0] <datetime.timedelta(days=2):
        vic.append(v)
print(vic)

['PN_EU_2_784', 'PN_EU_1_1193', 'PN_EU_0_1833', 'PN_EU_1_1855', 'PN_EU_0_336', 'PN_EU_0_323', 'PN_EU_0_452', 'PN_EU_0_364', 'PN_EU_0_295', 'PN_EU_0_525', 'PN_EU_0_462', 'PN_EU_0_33']
